In [67]:
import os
import sys
import numpy, scipy, matplotlib.pyplot as plt, sklearn, librosa, urllib, IPython.display
plt.rcParams['figure.figsize'] = (14,4)



In [111]:
def create_song_list(walk_dir):
    x = {}
    #c = 0
    for root, subdirs, files in os.walk(walk_dir):
        for filename in files:
            if ".mp3" in filename:
                file_path = os.path.join(root, filename)
                artist=root.split("\\")[1]
                song=filename.split(" ",1)[1].split(".")[0]
                if artist not in x:
                    x[artist] = []
                x[artist].append((song,file_path))
                #print(file_path)
                #x.append({"artist": artist,"song": song,"path":file_path})
                #a, fs = librosa.load(file_path)
                #mfcc = librosa.feature.mfcc(a, sr=fs).t
                #x.append({"artist": artist,"song": song,"mfcc":mfcc})
    return(x)

In [113]:
x=create_song_list("canzoni")
x

{'Ed Sheeran': [('Eraser', 'canzoni\\Ed Sheeran\\÷ [Deluxe]\\01 Eraser.mp3'),
  ('Castle On The Hill',
   'canzoni\\Ed Sheeran\\÷ [Deluxe]\\02 Castle On The Hill.mp3'),
  ('Dive', 'canzoni\\Ed Sheeran\\÷ [Deluxe]\\03 Dive.mp3'),
  ('Shape Of You', 'canzoni\\Ed Sheeran\\÷ [Deluxe]\\04 Shape Of You.mp3'),
  ('Perfect', 'canzoni\\Ed Sheeran\\÷ [Deluxe]\\05 Perfect.mp3'),
  ('Galway Girl', 'canzoni\\Ed Sheeran\\÷ [Deluxe]\\06 Galway Girl.mp3'),
  ('Happier', 'canzoni\\Ed Sheeran\\÷ [Deluxe]\\07 Happier.mp3'),
  ('New Man', 'canzoni\\Ed Sheeran\\÷ [Deluxe]\\08 New Man.mp3'),
  ("Hearts Don't Break Around Here",
   "canzoni\\Ed Sheeran\\÷ [Deluxe]\\09 Hearts Don't Break Around Here.mp3"),
  ('What Do I Know-',
   'canzoni\\Ed Sheeran\\÷ [Deluxe]\\10 What Do I Know-.mp3'),
  ('How Would You Feel (Paean)',
   'canzoni\\Ed Sheeran\\÷ [Deluxe]\\11 How Would You Feel (Paean).mp3'),
  ('Supermarket Flowers',
   'canzoni\\Ed Sheeran\\÷ [Deluxe]\\12 Supermarket Flowers.mp3'),
  ('Barcelona', 'canzon

In [103]:
path=x[0]["path"]
path
#x, fs = librosa.load('canzoni\\Ed Sheeran\\÷ [Deluxe]\\01 Eraser.mp3')

'canzoni\\Ed Sheeran\\÷ [Deluxe]\\01 Eraser.mp3'